In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

- analyze results +
- Normalize 
- add lag
- try other models +
- tune hyperparameters
    - cheby k
    - filter out
    - nn depth
    - nn width
    - non-linear filter function
    - learning rate
    - optimizer
    - epochs
- try dynamic graph split
- incorporate storage_location,product_group

In [2]:
G = pickle.load(open('graphs/graph.pkl','rb'))

In [3]:
d2d = pd.read_csv('Raw Dataset/Homogenoeus/Temporal Data/Weight/Delivery To distributor.csv')
factory_issue = pd.read_csv('Raw Dataset/Homogenoeus/Temporal Data/Weight/Factory Issue.csv')
production = pd.read_csv('Raw Dataset/Homogenoeus/Temporal Data/Weight/Production .csv')
sales_orders = pd.read_csv('Raw Dataset/Homogenoeus/Temporal Data/Weight/Sales Order .csv')

In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

d2d = scaler.fit_transform(d2d.drop('Date',axis=1))
factory_issue = scaler.fit_transform(factory_issue.drop('Date',axis=1))
sales_orders = scaler.fit_transform(sales_orders.drop('Date',axis=1))

In [5]:
d2d[:0].shape

(0, 41)

In [6]:
columns = production.drop('Date',axis=1).columns
for node in G.nodes:
    node_mask = [n == node for n in columns] 
    G.nodes[node]['delivered'] = d2d[:,node_mask]
    G.nodes[node]['factory issue'] = factory_issue[:,node_mask]
    G.nodes[node]['sales_order'] = sales_orders[:,node_mask]

In [7]:
G.nodes['SOS008L02P']['production'].shape

(221,)

In [8]:
G_processed = nx.convert_node_labels_to_integers(G)


In [9]:
G_processed.remove_edges_from([(u,v) for u,v,d in G_processed.edges(data=True) if d['Plant'] == 0])

In [10]:
edge_attr = np.zeros((len(G_processed.edges()),1))

for i,e in enumerate(G_processed.edges(data=True)):
    # edge_attr[i] = np.array(list(e[2].values()),dtype=float)
    edge_attr[i] = list(e[2].values())[0]

#edge_attr = edge_attr.transpose()


In [11]:
d2d = pd.read_csv('Raw Dataset/Homogenoeus/Temporal Data/Weight/Delivery To distributor.csv')

In [12]:
duration = len(d2d)
node_feature_sequence = []
node_target_sequence = []
for i in range(duration-1):
    node_features = []
    node_targets = []
    for n,d in G_processed.nodes(data=True):
        node_features.append([d['delivered'][i][0],d['sales_order'][i][0],d['factory issue'][i][0]])
        node_targets.append(d['production'][i+1])
    node_feature_sequence.append(np.array(node_features))
    node_target_sequence.append(np.array(node_targets))
    
    

In [13]:
np.shape(node_feature_sequence)

(220, 40, 3)

In [14]:
np.shape(node_target_sequence)

(220, 40)

In [15]:
np.array(list(G_processed.edges())).transpose().max()

39

In [ ]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal,DynamicGraphTemporalSignal

data_iter = StaticGraphTemporalSignal(
    edge_index= np.array(list(G_processed.edges())).transpose(),
    edge_weight = edge_attr,
    features = node_feature_sequence,
    targets = node_target_sequence
)

In [ ]:
from torch_geometric_temporal.signal import temporal_signal_split

train, test = temporal_signal_split(data_iter, train_ratio=0.8)

In [ ]:
for time, snapshot in enumerate(train):
    print(snapshot.x.shape)
    print(snapshot.edge_index.shape)
    print(snapshot.edge_attr.shape)
    print(snapshot.num_nodes)
    print(snapshot.num_edge_features)
    print(snapshot.y.shape)
    break

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU

class GRU(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(GRU, self).__init__()
        self.recurrent = GConvGRU(node_features, filters, 3)
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [ ]:
from tqdm import tqdm

model = GRU(node_features=3,filters=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(100)):
    for time, snapshot in enumerate(train):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = torch.mean((y_hat-snapshot.y)**2)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()


In [ ]:
model.eval()
pred = []
true = []
cost = 0
for time, snapshot in enumerate(test):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    pred.append(y_hat.detach().numpy())
    true.append(snapshot.y)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("RMSE: {:.4f}".format(cost**0.5))

In [ ]:
221 - 45

In [ ]:
pred_r = np.reshape(pred,np.shape(true))
true = np.array(true)
node_names = [n  for n in G.nodes]
for i in range(40):
    node = node_names[i]
    plt.plot(production[node])
    plt.plot(x = range(176,221),y= pred_r[:,i])
    plt.plot(x = range(176,221),y=true[:,i])
    plt.show()